<a href="https://colab.research.google.com/github/datarobot-community/custom-models/blob/master/drum_overview/Main_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using MLOps DRUM to test your custom models
**Author**: Tim Whittaker

#### Scope
We'll get our hands dirty by:

* Building a simple regression model using Scikit
* Using DRUM for Batch Scoring
* Using DRUM to get a REST API endpoint
* Show a simple example app connected to the REST API
* H2O, Keras, XGBoost, and DataRobot
* Add a DataRobot remote agent if you are interested in further model monitoring



In [1]:
#Clone the repository
!git clone https://github.com/datarobot-community/custom-models

fatal: destination path 'custom-models' already exists and is not an empty directory.


In [2]:
!pip install -r ~/custom-models/drum_overview/requirements.txt

     |████████████████████████████████| 1.2 MB 1.9 MB/s eta 0:00:01


  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.2.1
    Uninstalling xgboost-1.2.1:
      Successfully uninstalled xgboost-1.2.1


## Train a regression model

A simple RandomForestRegressor to predict house prices in Boston.

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import pickle
import datetime

# Read the train and test data
TRAIN_DATA_REG = "./custom-models/drum_overview/data/boston_housing_train.csv"  # 14 features
TEST_DATA_REG = "./custom-models/drum_overview/data/boston_housing_test.csv"  # 13 features - target is removed

reg_X_train = pd.read_csv(TRAIN_DATA_REG)
reg_Y_train = reg_X_train.pop('MEDV')

reg_X_test = pd.read_csv(TEST_DATA_REG)

# Fit the model
rf = RandomForestRegressor()
rf.fit(reg_X_train, reg_Y_train)

# Pickle the file and write it to the file system
with open("./custom-models/drum_overview/custom_model_reg/reg_rf_model.pkl", 'wb') as pkl:
    pickle.dump(rf, pkl)
    
# Call predict to confirm it works
rf.predict(reg_X_test)

array([25.703, 21.949, 34.722, 33.492, 35.367, 26.658, 21.711, 23.087,
       17.043])

## Testing the Model

Input the prediction dataset that includes all features except the target feature.

*Note if you see the error `xgboost.core.XGBoostError: XGBoost Library (libxgboost.dylib) could not be loaded` when trying to run the performance test below, you will need to install the libomp package via homebrew as in the cell below.*

In [6]:
!brew install libomp

==> Downloading https://ghcr.io/v2/homebrew/core/libomp/manifests/12.0.1
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/libomp/blobs/sha256:cba5086bd24
==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sh
######################################################################## 100.0%
==> Pouring libomp--12.0.1.big_sur.bottle.tar.gz
🍺  /usr/local/Cellar/libomp/12.0.1: 9 files, 1.5MB


In [2]:
%%sh 
drum perf-test --code-dir ~/custom-models/drum_overview/custom_model_reg --input ~/custom-models/drum_overview/data/boston_housing_test.csv  --target-type regression

DRUM performance test
Model:      /Users/melanie.hanna/custom-models/drum_overview/custom_model_reg
Data:       /Users/melanie.hanna/custom-models/drum_overview/data/boston_housing_test.csv
# Features: 13
Preparing test data...



Running test case with timeout: 600
Running test case: 72 bytes - 1 samples, 100 iterations
Running test case with timeout: 600
Running test case: 0.1MB - 1449 samples, 50 iterations
Running test case with timeout: 600
Running test case: 10MB - 144964 samples, 5 iterations
Running test case with timeout: 600
Running test case: 50MB - 724823 samples, 1 iterations
Test is done stopping drum server

  size     samples   iters    min     avg     max     total     used     total p
                                                       (s)      (MB)     hysical
                                                                          (MB)  
72 bytes         1     100   0.014   0.023   0.093     2.274   350.738   16384.0
                                             

tput: terminal attributes: Device not configured



## Validating the Model

In [3]:
%%sh 
drum validation --code-dir ~/custom-models/drum_overview/custom_model_reg \
--input ~/custom-models/drum_overview/data/boston_housing_test.csv \
--target-type regression > drum_validation.log

In [4]:
%%sh
tail drum_validation.log



Validation checks results
      Test case          Status   Details
Basic batch prediction   PASSED          
Null value imputation    PASSED          


# Batch Scoring with DRUM
<a id="setup_complete"></a>

At this point our model has been written to disk and we want to start making predictions with it.  To do this, we'll leverage DRUM and it's ability to natively handle our scikit learn model, all we need to do is tell DRUM where it resides as well as the data we wish to score.  

There are a lot of frameworks which DRUM supports natively, but for those which DRUM doesn't support off the shelf, we'll just need to create some custom hooks so DRUM.  In this example, we'll highlight some very simple custom hooks, and will provide links to more complex examples.  

In [5]:
%%sh 
drum score --code-dir ~/custom-models/drum_overview/custom_model_reg \
--input ~/custom-models/drum_overview/data/boston_housing_test.csv \
--output ~/custom-models/drum_overview/data/predictions.csv --target-type regression

In [6]:
pd.read_csv("~/custom-models/drum_overview/data/predictions.csv").head()

,Predictions
0,25.941
1,22.129
2,34.559
3,33.845
4,35.273


# Start the inference server locally

Batch scoring can be very useful, but the utility DRUM offers does not stop there.  We can also leverage DRUM to serve our model as a RESTful API endpoint.  The only thing that changes is the way we will structure the command - using the `server` mode instead of `score` model.  We'll also need to provide an address which is NOT in use.  

When starting the server, we'll use `subprocess.Popen` so we may interact with the server in this notebook

In [7]:
import subprocess
import requests
import pandas as pd
from io import BytesIO
import yaml
import time
import os
import datarobot as dr
from pprint import pprint

In [50]:
PORT = '6001'

In [51]:
run_inference_server = ["drum",
              "server",
              "--code-dir","./custom-models/drum_overview/custom_model_reg", 
              "--address", f"0.0.0.0:{PORT}", 
              "--show-perf",
              "--target-type", "regression",
              "--logging-level", "info",
              "--show-stacktrace",
              "--verbose"
              ]

In [52]:
inference_server = subprocess.Popen(run_inference_server, stdout=subprocess.PIPE)

In [53]:
## confirm the server is running
time.sleep(10) ## snoozing before pinging the server to give it time to actually start
print('check status')
requests.request("GET", f"http://0.0.0.0:{PORT}").content

check status


b'{"message":"OK"}\n'

## Send data to server for inference

The request must provide our dataset as form data.  In order to do so, we'll create a simple python function to pass the data over appropriately.  We'll leverage the same function in our simple flask app a little later.  

In [37]:
def score(data, port = PORT):
    b_buf = BytesIO()
    b_buf.write(data.to_csv(index=False).encode("utf-8"))
    b_buf.seek(0)
  
    url = "http://localhost:{}/predict/".format(port)
    files = [
        ('X', b_buf)
    ]
    response = requests.request("POST", url, files = files, timeout=None, verify=False)
    return response

In [38]:
# %%timeit
scoring_data = pd.read_csv("~/custom-models/drum_overview/data/boston_housing_test.csv")
predictions = score(scoring_data).json() ## score entire dataset but only show first 5 records
pprint(predictions)

{'predictions': [25.703,
                 21.949,
                 34.722,
                 33.492,
                 35.367,
                 26.658,
                 21.711,
                 23.087,
                 17.043]}


In [39]:
requests.request("GET", f"http://0.0.0.0:{PORT}/").content

b'{"message":"OK"}\n'

In [54]:
inference_server.terminate()
inference_server.stdout.readlines()

[b'Detected REST server mode - this is an advanced option\n',
 b'Detected /Users/melanie.hanna/custom-models-wip/drum_overview/custom-models/drum_overview/custom_model_reg/custom.py .. trying to load hooks\n',
 b'\x1b \x1b\n',
 b'\x1b \x1b\n',
 b'\x1b============================================================\x1b\n',
 b'\x1bComponent: Prediction Server\x1b\n',
 b'\x1bOutput:\x1b\n',
 b'\x1b------------------------------------------------------------\x1b\n']

In [55]:
inference_server.kill()

In [56]:
#Stop the flask server
!lsof -i tcp:6001 | grep -v PID | awk '{print $2}' | xargs kill

## Value Prop

One may ask, what is the benefit to be had here?  Well, first of, there is not need for me to write an api to get the model up and running.  Second, DRUM allows me to abstract the framework away (provided I'm using one that is natively supported, or I can write enough python so that DRUM understands how to hook up to the model.  

For example, I could hot swap models as I see fit (see exampels in `./src/other_models`)

While we will run through several other frameworks with in `score` you can bet they are supported in `server` mode as well!

#### H2o Mojo

In [57]:
!drum score --code-dir ~/custom-models/drum_overview/other_models/h2o_mojo/regression --input ~/custom-models/drum_overview/data/boston_housing_test.csv --target-type regression

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.


java gateway failed to start


#### Keras

In [58]:
!drum score --code-dir ~/custom-models/drum_overview/other_models/python3_keras_joblib --input ~/custom-models/drum_overview/data/boston_housing_test.csv --target-type regression

/Users/melanie.hanna/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator SimpleImputer from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/melanie.hanna/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/melanie.hanna/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Pipeline from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/melanie.hanna/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator ColumnTransformer from version 0.23.1 whe

#### XGBoost

In [59]:
!drum score --code-dir ~/custom-models/drum_overview/other_models/python3_xgboost --input ~/custom-models/drum_overview/data/boston_housing_test.csv --target-type regression

   Predictions
0    24.541843
1    21.260277
2    34.018497
3    32.569200
4    34.248066
5    27.282364
6    20.803959
7    19.645220
8    16.968882


#### DataRobot Codegen

In [60]:
!drum score --code-dir ~/custom-models/drum_overview/other_models/dr_codegen --input ~/custom-models/drum_overview/data/boston_housing_test.csv --target-type regression

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.


java gateway failed to start
